In [35]:
import requests
import os
import pandas as pd
import json
from tqdm import tqdm

index_dir='library_index.csv'


headers = {'x-api-key': api_key}


## get paper id

In [36]:

match_api='https://api.semanticscholar.org/graph/v1/paper/search/match?query={title}'

index_csv=pd.read_csv(index_dir)
index_dict=index_csv.to_dict(orient='records')
index={}
for i in index_dict:
    index[i['acronym'].lower()]=i['title']


In [37]:
papers={}
for i in tqdm(index):
    papers[i]=requests.get(match_api.format(title=index[i]), headers=headers).json()

100%|██████████| 262/262 [02:45<00:00,  1.58it/s]


In [38]:
index_csv

,acronym,title,id
0,transformer,Attention is All you Need,204e3073870fae3d05bcbc2f6a8e263d9b72e776
1,gpt,Improving Language Understanding by Generative...,cd18800a0fe0b668a1cc19f2ec95b5003d0a5035
2,gpt2,Language Models are Unsupervised Multitask Lea...,9405cc0d6169988371b2755e573cc28650d14dfe
3,gpt3,Language Models are Few-Shot Learners,90abbc2cf38462b954ae1b772fac9532e2ccd8b0
4,bert,BERT: Pre-training of Deep Bidirectional Trans...,df2b0e26d0599ce3e70df8a9da02e51594e0e992
...,...,...,...
257,reparamdiscdiffu,A Reparameterized Discrete Diffusion Model for...,1f898d66acabff511a3871b82799aa73c0055402
258,derandddm,Fast Sampling via Discrete Non-Markov Diffusio...,abca18159e0836406a648414cd4275715f3cc12e
259,dinoiser,DINOISER: Diffused Conditional Sequence Learni...,020a50f6a7154850ac81e3cde69ad8198ded6751
260,sgconv,What Makes Convolutional Models Great on Long ...,240300b1da360f22bf0b82c6817eacebba6deed4


In [27]:
index_csv['acronym']=index_csv['acronym'].str.lower()

In [40]:
unmatched=[]

for i in papers:
    try:
        id=papers[i]['data'][0]['paperId']
        index_csv.loc[index_csv['acronym']==i,'id']=id
    except:
        unmatched.append(i)
        print('Error:',i)

print(len(unmatched))

Error: ntk
Error: longnet
Error: feedbackmem
Error: srt
Error: kangpt
5


In [41]:

for i in tqdm(unmatched):
    papers[i]=requests.get(match_api.format(title=index[i])).json()

100%|██████████| 5/5 [00:02<00:00,  2.20it/s]


## Construct metadata & references

In [42]:
meta={}
meta_dir='library_meta.json'
if os.path.exists(meta_dir):
    with open(meta_dir,'r') as f:
        meta=json.load(f)

paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=year,authors,tldr,venue,abstract,citationCount,influentialCitationCount,references,embedding.specter_v2,openAccessPdf'

for i in tqdm(index_csv.index):
    acronym=index_csv.loc[i,'acronym']
    title=index_csv.loc[i,'title']
    if 'detail' in meta.get(acronym,{}):
        if not 'message' in meta[acronym]['detail']:
            continue
    if not pd.isna(index_csv.loc[i,'id']):
        paper_id=index_csv.loc[i,'id']
        detail=requests.get(paper_detail.format(paper_id=paper_id)).json()
        if 'message' in detail:
            print('Error:',acronym,detail['message'])
            continue
        meta[acronym]={}
        meta[acronym]['title']=title
        meta[acronym]['id']=paper_id
        meta[acronym]['detail']=detail
    else:
        meta[acronym]={}
        meta[acronym]['title']=title
        meta[acronym]['id']=paper_id
        meta[acronym]['detail']={}

with open(meta_dir,'w') as f:
    json.dump(meta,f)
        

references={}
ref_dir='library_ref.json'
if os.path.exists(ref_dir):
    with open(ref_dir,'r') as f:
        references=json.load(f)

references_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}/references?fields=contextsWithIntent,intents,isInfluential,title,influentialCitationCount'

for i in tqdm(index_csv.index):
    acronym=index_csv.loc[i,'acronym']
    id=index_csv.loc[i,'id']
    if acronym in references:
        if 'message' not in references[acronym]:
            continue
    if pd.isna(id):
        references[acronym]=[]
    else:
        ret=requests.get(references_detail.format(paper_id=id)).json()
        if 'message' in ret:
            print('Error:',acronym,ret['message'])
            continue
        references[acronym]=ret
        

with open(ref_dir,'w') as f:
    json.dump(references,f)


100%|██████████| 262/262 [00:00<00:00, 950.51it/s] 


## Metadata to tree

In [43]:
index=pd.read_csv('./library_index.csv')
with open('./library_meta.json','r') as f:
    meta=json.load(f)
with open('./library_ref.json','r') as f:
    refs=json.load(f)

In [94]:
meta['transformer']

{'title': 'Attention is All you Need',
 'id': '204e3073870fae3d05bcbc2f6a8e263d9b72e776',
 'detail': {'paperId': '204e3073870fae3d05bcbc2f6a8e263d9b72e776',
  'abstract': 'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-

In [44]:
from dataclasses import dataclass, field, asdict
from typing import List, Dict


LIBRARY_DIR = './'

@dataclass
class PaperObject:
    title: str
    acronym: str
    s2id: str
    abstract: str
    venue: str
    year: int
    tldr: str
    # embedding: list
    citationCount: int
    influentialCitationCount: int
    seed_ids: List[str]
    code: str = None

    def __post_init__(self):
        code_dir=os.path.join(LIBRARY_DIR,'base',self.acronym,self.acronym+'_edu.py')
        if os.path.exists(code_dir):
            self.code=open(code_dir,'r').read()
        else:
            self.code=None

    def to_dict(self) -> Dict:
        return asdict(self)
    
    @property
    def type(self) -> str:
        return self.__class__.__name__

    @classmethod
    def from_dict(cls, dict: Dict):
        return cls(**dict)
    
    @classmethod
    def load(cls, tree_dir: str, id:str):
        with open(os.path.join(tree_dir,id+'.json'),'r') as f:
            return cls.from_dict(json.load(f))

    def save(self,tree_dir: str):
        os.makedirs(tree_dir, exist_ok=True)
        with open(os.path.join(tree_dir,self.acronym+'.json'),'w') as f:
            json.dump(self.to_dict(),f,indent=4)

    def to_desc(self) -> str:
        title=self.title.replace(':',' ')
        abstract=self.abstract.replace(':',' ')
        tldr=self.tldr.replace(':',' ')
        venue=self.venue.replace(':',' ')
        mdtext=f'# {title}\nS2 ID {self.s2id}\n\nTL;DR {tldr}\n\n{abstract}\n\nPublished at {venue} in {self.year}\n\nCited {self.citationCount} times\n\nImpactful citations: {self.influentialCitationCount}'
        return mdtext


for acronym in refs:
    if refs[acronym]==[]:
        continue   
    refdata=refs[acronym]['data']
    seed_ids=[]
    for ref in refdata:
        if 'methodology' in ref['intents']:
            if ref['citedPaper']['paperId'] in index['id'].values:
                ref_acronym=index.loc[index['id']==ref['citedPaper']['paperId'],'acronym'].values[0]
                seed_ids.append(ref_acronym)
    title=meta[acronym]['title']
    s2id=meta[acronym]['id']
    abstract=meta[acronym]['detail']['abstract']
    if abstract is None:
        abstract='N/A'
    venue=meta[acronym]['detail']['venue']
    if venue is None:
        venue='N/A'
    year=meta[acronym]['detail']['year']
    if year is None:
        year='N/A'
    tldr=meta[acronym]['detail']['tldr']
    if tldr is None:
        tldr='N/A'
    else:
        tldr=tldr['text']
        if tldr is None: tldr='N/A'
    embedding=meta[acronym]['detail']['embedding']
    if embedding is None:
        embedding=[]
    else:
        embedding=embedding['vector']
    citationCount=meta[acronym]['detail']['citationCount']
    influentialCitationCount=meta[acronym]['detail']['influentialCitationCount']
    paper=PaperObject(title,acronym,s2id,abstract,venue,year,tldr,citationCount,influentialCitationCount,seed_ids)
    paper.save('./tree')


## Build 1 hoc impactful cites

In [45]:
index=pd.read_csv('./library_index.csv')
with open('./library_meta.json','r') as f:
    meta=json.load(f)
dir_1hoc='./expanded_tree'
get_cite='https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations?fields=intents,contextsWithIntent,isInfluential,title&offset={offset}&limit=1000'
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=year,authors,tldr,venue,abstract,citationCount,influentialCitationCount,references,embedding.specter_v2,openAccessPdf'

In [46]:
import time

cite_1hoc_dir=dir_1hoc+'/cite_1hoc.json'
detail_1hoc_dir=dir_1hoc+'/detail_1hoc.json'
os.makedirs(dir_1hoc,exist_ok=True)

cite_1hoc={}
detail_1hoc={}
if os.path.exists(cite_1hoc_dir):
    with open(cite_1hoc_dir,'r') as f:
        cite_1hoc=json.load(f)
if os.path.exists(detail_1hoc_dir):
    with open(detail_1hoc_dir,'r') as f:
        detail_1hoc=json.load(f)

for acronym in tqdm(index['acronym']):
    id=index.loc[index['acronym']==acronym,'id'].values[0]
    if pd.isna(id):
        continue
    citecount=meta[acronym]['detail']['citationCount']
    if acronym in cite_1hoc:
        if 'message' not in cite_1hoc[acronym]:
            print('Already done:',acronym,len(cite_1hoc[acronym]),citecount)
            continue
    cite_1hoc[acronym]=[]
    for offset in range(0,9001,1000):
        if offset+1000>=10000:
            offset=8999
        print(acronym,offset,offset+1000)
        cites=requests.get(get_cite.format(paper_id=id,offset=offset),headers=headers).json()
        if 'message' in cites:
            print('Error:',id,cites['message'])
            continue
        if 'data' not in cites:
            print('Error:',cites)
            raise
        for c in cites['data']:
            paperid=c['citingPaper']['paperId']
            if paperid in cite_1hoc[acronym]:
                continue
            if 'methodology' in c['intents'] and c['isInfluential']:
                cite_1hoc[acronym].append(paperid)
        time.sleep(0.1)
    print('Done',acronym,len(cite_1hoc[acronym]),citecount)
    with open(cite_1hoc_dir,'w') as f:
        json.dump(cite_1hoc,f)



  0%|          | 0/262 [00:00<?, ?it/s]

Already done: transformer 684 98603
Already done: gpt 789 28375
Already done: gpt2 1162 17403
Already done: gpt3 587 28360
Already done: bert 1101 78188
Already done: ttt 0 2
Already done: xlstm 1 14
Already done: griffin 6 48
Already done: hyena 27 163
Already done: m2 3 22
Already done: spikegpt 4 54
Already done: mamba2 3 25
Already done: s4 102 744
Already done: hippo 34 255
Already done: lssl 7 244
Already done: httyh 6 50
Already done: s4d 32 166
Already done: mamba 175 662
Already done: samba 1 5
Already done: retnet 16 143
Already done: gla 9 43
Already done: rebased 0 0
Already done: deltanet 21 151
Already done: hedgehog 0 13
Already done: polysketchformer 0 0
Already done: transnormerllm 2 7
Already done: rwkv4 23 251
Already done: gateloop 0 12
Already done: abc 4 21
Already done: vqtransformer 0 5
Already done: hgrn 5 38
Already done: hgrn2 0 11
Already done: rwkv6 1 16
Already done: mega 23 121
Already done: dcmha 0 0
Already done: lllm 0 1
Already done: ring 5 68
Already

100%|█████████▉| 261/262 [00:02<00:00, 125.70it/s]

Done sgconv 11 69
liquids4 0 1000
liquids4 1000 2000
liquids4 2000 3000
liquids4 3000 4000
liquids4 4000 5000
liquids4 5000 6000
liquids4 6000 7000
liquids4 7000 8000
liquids4 8000 9000
liquids4 8999 9999


100%|██████████| 262/262 [00:09<00:00, 28.72it/s] 

Done liquids4 4 55


In [47]:

for acronym in tqdm(cite_1hoc):
    for c in cite_1hoc[acronym]:
        if c in detail_1hoc:
            if 'message' not in detail_1hoc[c]:
                continue
        detail=requests.get(paper_detail.format(paper_id=c)).json()
        if 'message' in detail:
            print('Error:',c,detail['message'])
            continue
        detail_1hoc[c]=detail
        time.sleep(0.1)
    print('Done',acronym,len(detail_1hoc),len(cite_1hoc[acronym]))
    with open(detail_1hoc_dir,'w') as f:
        json.dump(detail_1hoc,f)

Error: dd3491a632e4bca6e67b17caf8e831248e958689 Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form
Error: 49653c3e849a5e403949ba0545c70cf90664a163 Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form
Error: 90fe81e0c94e111482741c4c0251c3be673f8c24 Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form
Error: 564d2c79772f3e1e847fa8c410f5ab2ad6134c8d Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form
Error: 83216ea36ae0f492d12082c8e8565b3143123eca Too Many Requests. Please wait and try again or apply for a key for higher rate limits. https://www.semanticscholar.org/product/api#api-key-form
Error: c7b01581b571f958059655468169